# The Task

As per the problem statement, I am required to return the closest questions as per a user's query. Thus in order to understand and learn from the data, I need to gather Questions and Answers that were posted on Stack Overflow. Thus what I need are the following: 

- Title 
- Question body
- Answers for that question
- Votes for each answers

Due to the sheer abundance of data on Stack Overflow and better sanity checks, I restricted the data to only "Python" related questions. However, the entire process in reproducible for other topics as well

# Data Collection

**Google BigQuery** dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer. More info about the dataset is given at: https://www.kaggle.com/stackoverflow/stackoverflow

- **bq_helper** is a helper class to perform read-only BigQuery Tasks
- There are many tables on the Stackoverflow database, but we only need concern ourselves with **posts_questions** and **posts_answers**

The query the retrive the required data is *SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score FROM 'bigquery-public-data.stackoverflow.posts_questions' AS q INNER JOIN 'bigquery-public-data.stackoverflow.posts_answers' AS a ON q.id = a.parent_id WHERE q.tags LIKE '%python%' LIMIT 500000*

1. **SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score**: Retreives the required columns for the data set
2. **'bigquery-public-data.stackoverflow.posts_questions' AS q INNER JOIN 'bigquery-public-data.stackoverflow.posts_answers' AS a ON q.id = a.parent_id**: This joins the two tables based on the Question ID. Question ID is stored as **id** in "posts_questions" and as **parent_id** in "posts_answers"
3. **WHERE q.tags LIKE '%python%'**: To restrict data to only python related questions
4. **LIMIT 500000**: In order to collect not more than 500,000 data points for faster processing 

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Database Analysis and Pre-processing

## Load Data

In [6]:
#  pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.0.0/en_core_web_sm-2.0.0.tar.gz#en_core_web_sm
# python3 -m spacy download en_core_web_sm-2.2.0 --direct
# TODO: en_core_web_sm and thinc causing issues
import en_core_web_sm
import thinc
import pandas as pd
import numpy as np
import spacy
from nltk import RegexpTokenizer
# import spacy.en_core_web_sm
# EN = en_core_web_sm.load()
# spacy.download('en')
EN = spacy.load('en_core_web_sm')
df = pd.read_csv('/content/gdrive/My Drive/Stackoverflow_VS_extension/Data.csv')
df.head()

,Unnamed: 0,id,title,body,tags,answers,score
0,0,8533318,"multiprocessing.Pool: When to use apply, apply...",<p>I have not seen clear examples with use-cas...,python|multithreading|concurrency|multiprocessing,"<p>Back in the old days of Python, to call a f...",435
1,1,5599022,Python: Pass a generic dictionary as a command...,<p>I have a script that needs to take in the n...,python|command-line-arguments,<p>That should be fairly easy to parse yoursel...,17
2,2,5178416,libxml install error using pip,<p>This is my error:</p>\n\n<pre><code>(mysite...,python|lxml|pip,<p>This worked for me:</p>\n\n<pre><code>yum i...,108
3,3,5178416,libxml install error using pip,<p>This is my error:</p>\n\n<pre><code>(mysite...,python|lxml|pip,<blockquote>\n <p>** make sure the developmen...,486
4,4,1995734,How are exceptions implemented under the hood?,"<p>Just about everyone uses them, but many, in...",c++|python|c|exception,<p>In his book <em>C Interfaces and Implementa...,18


In [7]:
print('Datebase shape:' + str(df.shape))

Datebase shape:(500000, 7)


## Missing Values

No missing values detected in any of the columns

In [8]:
df.isna().sum()

Unnamed: 0    0
id            0
title         0
body          0
tags          0
answers       0
score         0
dtype: int64

## Database Manipulation

In order to construct a corpus, I grouped all the answers by concatenating them based on their common questions and tags. Moreover, I added the scores for each answer in order to get a collective score for an entire question

In [9]:
# concat answers for each post
grouped = df.groupby(['id','title', 'body','tags'], as_index = False).agg(
    combined_answers=pd.NamedAgg(column='answers', aggfunc=lambda x: "\n".join(x)),
    combined_score=pd.NamedAgg(column='score', aggfunc=np.sum))
deduped_df = pd.DataFrame(grouped)

In [10]:
deduped_df.head()

,id,title,body,tags,combined_answers,combined_score
0,337,XML Processing in Python,<p>I am about to build a piece of a project th...,python|xml,"<p><a href=""http://effbot.org/zone/element-ind...",97
1,502,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,python|windows|image|pdf,<p>ImageMagick delegates the PDF->bitmap conve...,55
2,683,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,python|arrays|iteration,<p>Using a list comprehension would build a te...,84
3,773,How do I use itertools.groupby()?,<p>I haven't been able to find an understandab...,python|itertools,<p>This basic implementation helped me underst...,946
4,1171,What is the most efficient graph data structur...,<p>I need to be able to manipulate a large (10...,python|performance|data-structures|graph-theory,"<p>Even though this question is now quite old,...",81


The following code block shows the result of combining answers and their scores

In [11]:
print('Max score before: ') 
print(np.max(df.score.values))

print('Max score after: ') 
print(np.max(deduped_df.combined_score.values))

Max score before: 
5316
Max score after: 
6963


A couple of helper functions for Text Preprocessing. The steps followed to process a piece of raw text are:

1. Convert raw text into tokens
2. Convert tokens to lower case
3. Remove punctuations
4. Remove Stopwords 

Note: I skipped removal of numeric data since I felt it would remove precious contextual information. I also skipped a 'Stemming/Lemmatization' step because I did not want alter the domain specific terms used in our corpus and risk losing precious information

In [12]:
import re
import nltk
# import inflect
nltk.download('stopwords')
from nltk.corpus import stopwords

def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**The raw text for Questions and Answers is given along with the HTML markup with which it was displayed on StackOverflow originally**. 
These refer usually to *p tags, h1-h6 tags and the code tags*

- I constructed a new feature column called 'post_corpus' by combining the title, question body, and all the answers
- I prepended the title to the question body 
- I skipped the 'code' sections because they do not offer useful information for our task
- I constructed urls for each question by appending 'https://stackoverflow.com/questions/' with the question id
- I constructed 2 features for sentiment using the open Source **Textblob library** 

In [13]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import lxml

title_list = [] 
content_list = []
url_list = []
comment_list = []
sentiment_polarity_list = []
sentiment_subjectivity_list = []
vote_list =[]
tag_list = []
corpus_list = []
for i, row in deduped_df.iterrows():
    title_list.append(row.title)    # Get question title
    tag_list.append(row.tags)     # Get question tags
    
    # Questions
    content = row.body
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()     # Remove the code section
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
        
    content_list.append(str(row.title) + ' ' + str(text))   # Append title and question body data to the updated question body
    
    url_list.append('https://stackoverflow.com/questions/' + str(row.id))
    
    # Answers
    content = row.combined_answers
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    comment_list.append(text)
    
    vote_list.append(row.combined_score)       # Append votes
    
    corpus_list.append(content_list[-1] + ' ' + comment_list[-1])     # Combine the updated body and answers to make the corpus
    
    sentiment = TextBlob(row.combined_answers).sentiment
    sentiment_polarity_list.append(sentiment.polarity)
    sentiment_subjectivity_list.append(sentiment.subjectivity)

content_token_df = pd.DataFrame({'original_title': title_list, 'post_corpus': corpus_list, 'question_content': content_list, 'question_url': url_list, 'tags': tag_list, 'overall_scores':vote_list,'answers_content': comment_list, 'sentiment_polarity': sentiment_polarity_list, 'sentiment_subjectivity':sentiment_subjectivity_list})

In [14]:
content_token_df.head()
print(content_token_df.shape)

(294697, 9)


## Filter for only the most common tags

Each post has a variable number of different tags. In order to narrow down the vast choices for a more accurate model

In [15]:
content_token_df.tags = content_token_df.tags.apply(lambda x: x.split('|'))   # Convert raw text data of tags into lists

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}
for tags in content_token_df.tags:
    for tag in tags:
        if tag not in tag_freq_dict:
            tag_freq_dict[tag] = 0
        else:
            tag_freq_dict[tag] += 1

As a personal choice, I decided to with **20 most common tags**, which are shown below

In [16]:
import heapq
most_common_tags = heapq.nlargest(20, tag_freq_dict, key=tag_freq_dict.get)

In [17]:
most_common_tags

['python',
 'python-3.x',
 'pandas',
 'django',
 'python-2.7',
 'numpy',
 'list',
 'dataframe',
 'matplotlib',
 'dictionary',
 'regex',
 'tkinter',
 'tensorflow',
 'flask',
 'string',
 'csv',
 'arrays',
 'json',
 'selenium',
 'beautifulsoup']

The plan is to filter only the data which contains at least one of most_common_tags 

In [18]:
final_indices = []
for i,tags in enumerate(content_token_df.tags.values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>1:   # The minimum length for common tags should be 2 because 'python' is a common tag for all
        final_indices.append(i)

In [19]:
final_data = content_token_df.iloc[final_indices]

## Data Normalization

- I created a separate column for the 'processed_title' because I wanted to preserve the original title because I wanted to serve the original titles in the app
- I also normalized the numeric 'scores'

In [20]:
# import spacy
# EN = spacy.load('en_core_web_sm')

# Preprocess text for 'question_body', 'post_corpus' and a new column 'processed_title'
final_data.question_content = final_data.question_content.apply(lambda x: preprocess_text(x))
final_data.post_corpus = final_data.post_corpus.apply(lambda x: preprocess_text(x))
final_data['processed_title'] = final_data.original_title.apply(lambda x: preprocess_text(x))

# Normalize numeric data for the scores
final_data.overall_scores = (final_data.overall_scores - final_data.overall_scores.mean()) / (final_data.overall_scores.max() - final_data.overall_scores.min())

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [21]:
final_data.tags = final_data.tags.apply(lambda x: '|'.join(x))    # Combine the lists back into text data
final_data.drop(['answers_content'], axis=1)     # Remove the answers_content columns because it is alreaady included in the corpus

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,original_title,post_corpus,question_content,question_url,tags,overall_scores,sentiment_polarity,sentiment_subjectivity,processed_title
2,Using 'in' to match an attribute of Python obj...,using match attribute python objects array nt ...,using match attribute python objects array nt ...,https://stackoverflow.com/questions/683,python|arrays|iteration,0.011301,0.163567,0.568209,using match attribute python objects array
6,Python version of PHP's stripslashes,python version php stripslashes wrote piece co...,python version php stripslashes wrote piece co...,https://stackoverflow.com/questions/13454,python|string|escaping,0.001115,0.195000,0.519274,python version php stripslashes
7,Unicode vs UTF-8 confusion in Python / Django?,unicode vs utf8 confusion python django stumbl...,unicode vs utf8 confusion python django stumbl...,https://stackoverflow.com/questions/22149,python|django|unicode,0.006997,0.082857,0.403250,unicode vs utf8 confusion python django
24,Using Django time/date widgets in custom form,using django time date widgets custom form use...,using django time date widgets custom form use...,https://stackoverflow.com/questions/38601,python|django,0.041431,0.302423,0.599938,using django time date widgets custom form
32,Can parallel traversals be done in MATLAB just...,parallel traversals done matlab python using f...,parallel traversals done matlab python using f...,https://stackoverflow.com/questions/49307,python|arrays|matlab|for-loop,0.002837,0.358333,0.752381,parallel traversals done matlab python
...,...,...,...,...,...,...,...,...,...
294679,How can I insert spaces between words given a ...,insert spaces words given list lists coded let...,insert spaces words given list lists coded let...,https://stackoverflow.com/questions/63758260,python|list|dictionary|spacing,-0.000607,0.243050,0.873782,insert spaces words given list lists coded let...
294680,Django creates another media folder inside med...,django creates another media folder inside med...,django creates another media folder inside med...,https://stackoverflow.com/questions/63758482,python|django|python-imaging-library,-0.000176,0.055556,0.444444,django creates another media folder inside med...
294682,Options for deploying Flask app that continuou...,options deploying flask app continuously web s...,options deploying flask app continuously web s...,https://stackoverflow.com/questions/63758866,python|flask|heroku|web-scraping|web-applications,-0.000607,0.044898,0.545003,options deploying flask app continuously web s...
294683,"Delete ""nan"" in python list",delete nan python list new python simple quest...,delete nan python list new python simple quest...,https://stackoverflow.com/questions/63758902,python|list,-0.000750,0.138095,0.636310,delete nan python list


In [22]:
# Save the data
final_data.to_csv('/content/gdrive/My Drive/Stackoverflow_VS_extension/Preprocessed_data.csv', index=False)